# Stock Price Forecasting using Stacked LSTM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [ ]:
df = pd.read_csv('/kaggle/input/nyse/prices-split-adjusted.csv')

In [ ]:
df

In [ ]:
df = df[df['symbol']=='AAP']
df

In [ ]:
df_close = df.reset_index()['close']
df_close.tail()

In [ ]:
df_close.shape

In [ ]:
plt.plot(df_close)
plt.show()

### MinMax Scaling as LSTM is sensitive to scale of data

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
df_close = scaler.fit_transform(np.array(df_close).reshape(-1,1))

In [ ]:
df_close.shape

In [ ]:
plt.plot(df_close)
plt.show()

In [ ]:
train_size = int(len(df_close)*0.65)
test_size = len(df_close) - train_size
train_data = df_close[0:train_size,:]
test_data = df_close[train_size:,:]

In [ ]:
train_size, test_size

In [ ]:
plt.figure(figsize=[15,5])
plt.subplot(121)
plt.plot(train_data)
plt.title('Train Data')
plt.subplot(122)
plt.plot(test_data)
plt.title('Test Data')
plt.show()

## Model

In [ ]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        dataX.append(dataset[i:(i+time_step),0])
        dataY.append(dataset[(i+time_step),0])
    
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 100
train_x, train_y = create_dataset(train_data, time_step)
test_x, test_y = create_dataset(test_data, time_step)

In [ ]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

In [ ]:
train_x = train_x.reshape(train_x.shape[0],train_x.shape[1],1)
test_x = test_x.reshape(test_x.shape[0],test_x.shape[1],1)

In [ ]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(100,1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model_history = model.fit(train_x, train_y, 
                          validation_data=(test_x, test_y), 
                          epochs=100, batch_size=64)

In [ ]:
model_history.history.keys()

In [ ]:
plt.figure(figsize=[10,6])
plt.plot(model_history.history['loss'], label='train loss')
plt.plot(model_history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

## Forecasting

In [ ]:
train_predict = model.predict(train_x)
test_predict = model.predict(test_x)

In [ ]:
trainPredictPlot = np.empty_like(df_close)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[time_step:len(train_predict)+time_step,:] = train_predict

testPredictPlot = np.empty_like(df_close)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict) + (time_step*2) +1:len(df_close)-1, :] = test_predict

plt.figure(figsize=[12,8])
plt.plot(scaler.inverse_transform(df_close))
plt.plot(scaler.inverse_transform(trainPredictPlot))
plt.plot(scaler.inverse_transform(testPredictPlot))
plt.show()



## Predict next 30 days

In [ ]:
x_input = test_data[0:100].reshape(1,-1)
x_input.shape

In [ ]:
temp_input = list(x_input)
temp_input = temp_input[0].tolist()
temp_input

In [ ]:
output = []
days = 517
for i in range(days):
    print('start')
    x_input = np.array(temp_input[i:])
    print(f'{i} day input {x_input}')    
    x_input = x_input.reshape((1, time_step, 1))
    yhat = model.predict(x_input, verbose=0)
    print(yhat)
    temp_input.extend(yhat[0].tolist())
    output.extend(yhat[0].tolist())    
    

In [ ]:
type(output)

In [ ]:
outputnp = np.array(output)

In [ ]:
outputnp = outputnp.reshape(-1,1)
outputnp.shape

In [ ]:
testPredictPlot = np.empty_like(df_close)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict) + (time_step*2) +1:, :] = outputnp

plt.figure(figsize=[12,8])
plt.plot(scaler.inverse_transform(df_close))
plt.plot(scaler.inverse_transform(testPredictPlot))
plt.show()
